In [1]:
import sys, os, time
# %load_ext autoreload
# %autoreload 2
os.environ['FOUNDATION_RUN_MODE'] = 'jupyter'
os.environ['FOUNDATION_SAVE_DIR'] = '/is/ei/fleeb/workspace/chome/trained_nets'
os.environ['FOUNDATION_DATA_DIR'] = '/is/ei/fleeb/workspace/local_data'
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from IPython import display
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
import torchvision.models
import torchvision
from torch.utils.data import Dataset, DataLoader, TensorDataset
import gym
import numpy as np
%matplotlib notebook
# %matplotlib tk
import matplotlib.pyplot as plt
import imageio
import seaborn as sns
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
import foundation as fd
from foundation import models
from foundation import util
from foundation import train
#from foundation.util import replicate, Cloner

from hybrid import get_model, get_data

np.set_printoptions(linewidth=120, suppress=True)

In [2]:
M = train.Run_Manager(tbout='/is/ei/fleeb/workspace/trained_nets/managed')
M.clear_links()
M.start_tb()

/is/ei/fleeb/workspace/trained_nets/managed is available to view runs on tensorboard


/is/ei/fleeb/workspace/foundation/foundation/train/config.py:58: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f)


Found 258 runs
Tensorboard started: http://localhost:6006/


In [6]:
#M.show()
M.clear_filters().select(model='fdwae', dataset='3dshapes').filter(lambda r: not 'test' in r.name)
M.filter(lambda r: '200108' in r.name or '200109' in r.name or '200110' in r.name)
# M.filter(lambda r: '191218' in r.name)

# M.filter_idx(7, 5, 27, 23, 34, 16, 15, 13, 30, 28, 10, 12, 18, 21, 0, 1, 72, 73, 74, 67, 66, 57)


M.show()

  0 - 3dshapes-fdwae_0090-5965922-05_200108-160042
  1 - 3dshapes-fdwae_0092-5966021-05_200108-171822
  2 - 3dshapes-fdwae_0094-5966044-05_200108-175529
  3 - 3dshapes-fdwae_0095-5966045-05_200108-175832
  4 - 3dshapes-fdwae_0096-5966047-05_200108-175918
  5 - 3dshapes-fdwae_0097-5966051-05_200108-181229
  6 - 3dshapes-fdwae_0100-5966131-09_200109-133028
  7 - 3dshapes-fdwae_0100-5966131-10_200109-133027
  8 - 3dshapes-fdwae_0101-5966132-03_200109-133045
  9 - 3dshapes-fdwae_0102-5966133-09_200109-133120
 10 - 3dshapes-fdwae_0102-5966133-10_200109-133109
 11 - 3dshapes-fdwae_0122-5969012-03_200110-140612


In [7]:
ignore = None
ignore = ['disc_steps', '_type']
M.show_unique(ignore)

0) 3dshapes-fdwae_0090-5965922-05_200108-160042
	model.generator.full_latent_dim - 16 (_)

1) 3dshapes-fdwae_0092-5966021-05_200108-171822
	model.generator.full_latent_dim - 16 (_)

2) 3dshapes-fdwae_0094-5966044-05_200108-175529

3) 3dshapes-fdwae_0095-5966045-05_200108-175832
	model.latent_dim - 8 (16)

4) 3dshapes-fdwae_0096-5966047-05_200108-175918
	model.latent_dim - 32 (16)

5) 3dshapes-fdwae_0097-5966051-05_200108-181229
	model.generator.full_latent_dim - 16 (_)

6) 3dshapes-fdwae_0100-5966131-09_200109-133028
	model.latent_dim - 8 (16)
	model.generator.full_latent_dim - 8 (_)

7) 3dshapes-fdwae_0100-5966131-10_200109-133027
	model.latent_dim - 8 (16)
	model.generator.full_latent_dim - 8 (_)
	model.prob - 0.5 (1)

8) 3dshapes-fdwae_0101-5966132-03_200109-133045
	model.generator.full_latent_dim - 16 (_)
	model.prob - 0.5 (1)

9) 3dshapes-fdwae_0102-5966133-09_200109-133120
	model.generator.splits - 4 (2)
	model.generator.adain_latent_dim - 16 (8)
	model.generator.full_latent_dim 

In [8]:
# M.link('{idx}__{model}__{unique}')
M.link('{job}__{model}__{unique}')

Unable to get first event timestamp for run 0102-5966133-06__dwae__generator.splits=4__generator.adain_latent_dim=16__generator.full_latent_dim=16__prob=0p5: No event timestamp could be found


In [6]:
# M.link('{name}__{unique}')

In [6]:
# os.listdir(M.tbout)

In [9]:
M.clear_links()

In [8]:
M.tbout

'/is/ei/fleeb/workspace/trained_nets/managed'

In [9]:
M.active[0].keys()

odict_keys(['name', 'path', 'ckpt_path', 'config', 'base', 'diffs'])

In [10]:
M.active[0].config.info.job

{{'ID':'5966131', 'num':'100', 'ps':'7', '_ipython_canary_method_should_not_exist_':{{}}}}

In [15]:
M.active[0].config.name.split('_')[-1]

'0100-5966131-07'

In [7]:
root = os.environ['FOUNDATION_SAVE_DIR']
root

'/is/ei/fleeb/workspace/chome/trained_nets'

In [8]:
for r in M.active:
    n = r.name
    num = len(os.listdir(os.path.join(root, n)))
    if num > 100:
        print('{} : {}'.format(n, num))

3dshapes-dvae_0100-5966131-07_200109-133026 : 113
3dshapes-dvae_0101-5966132-01_200109-133045 : 114
3dshapes-dvae_0101-5966132-02_200109-133053 : 113
3dshapes-dvae_0102-5966133-07_200109-133118 : 113
3dshapes-dvae_0102-5966133-08_200109-133107 : 113
3dshapes-dvae_0122-5969012-01_200110-140610 : 113
3dshapes-dvae_0122-5969012-02_200110-140616 : 113
3dshapes-dwae_0094-5966044-04_200108-175514 : 113
3dshapes-dwae_0095-5966045-04_200108-175850 : 113
3dshapes-dwae_0096-5966047-04_200108-175913 : 113
3dshapes-dwae_0097-5966051-04_200108-181233 : 113
3dshapes-dwae_0100-5966131-05_200109-133027 : 113
3dshapes-dwae_0100-5966131-06_200109-133124 : 113
3dshapes-dwae_0101-5966132-00_200109-133044 : 113
3dshapes-dwae_0102-5966133-05_200109-133202 : 113
3dshapes-dwae_0102-5966133-06_200109-133106 : 113
3dshapes-dwae_0122-5969012-00_200110-140613 : 113
3dshapes-fdwae_0094-5966044-05_200108-175529 : 113
3dshapes-fdwae_0095-5966045-05_200108-175832 : 113
3dshapes-fdwae_0096-5966047-05_200108-175918 : 1

In [18]:
import json

In [19]:
json.loads('[10,2,3]')

[10, 2, 3]

In [9]:
saveroot = os.environ['FOUNDATION_SAVE_DIR']
saveroot

'/is/ei/fleeb/workspace/chome/trained_nets'

In [15]:
for name in os.listdir(saveroot):
    try:
        if 'checkpoint_100.pth.tar' not in os.listdir(os.path.join(saveroot, name)):
            print(name)
    except (FileNotFoundError, NotADirectoryError):
#         print('ERROR: {}'.format(name))
        pass

test-pycharm_200116-125210
3dshapes-fvae_0092-5966021-02_200108-171821
3dshapes-fdwae_0090-5965922-05_200108-160042
redball-wpp_0132-5971295-04_200113-165212
redball-vae_0133-5971579-01_200114-120657
redball-dwae_0133-5971579-05_200114-120701
celeba-dwae_0076-5920870-6_191213-201051
redball-fdwae_0132-5971295-08_200113-165209
3dshapes-vae_0089-5943513-01_191224-232610
redball-dvae_0134-5971580-06_200114-120742
3dshapes-wpp_0088-5943512-04_191224-232308
3dshapes-vae_0088-5943512-00_191224-232237
3dshapes-vae_0138-5986313-04_200116-114436
redball-fwae_0133-5971579-02_200114-120657
3dshapes-fdwae_0088-5943512-07_191224-232308
3dshapes-fwae_0090-5965922-01_200108-160043
3dscr-fdvae_0082-5923472-11_191218-100110
test-pycharm_200116-124838
3dshapes-fvae_0106-5966166-00_200109-154555
celeba-wgan_0076-5920870-4_191213-201050
3dshapes-wpp_0098-5966052-00_200108-181326
3dscr-vae_0078-5920873-1_191213-202211
3dshapes-wae_0138-5986313-00_200116-114549
3dshapes-wgan_0093-5966029-00_200108-172349
ce

In [ ]:
for name in os.listdir(saveroot):
    path = os.path.join(saveroot, name)
    if 'config.yml' in os.listdir(path):
        print(name,end=' ')
        with open(os.path.join(path, 'config.yml'), 'r') as f:
            data = yaml.load(f)
        break
#         if 'parents' in data:
#             del data['parents']
#             with open(os.path.join(path, 'config.yml'), 'w') as f:
#                 yaml.dump(data, f)
#             print('fixed')
#         else:
#             print()

In [ ]:
data['parents']